In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from harmonic_inference.data.corpus_reading import aggregate_annotation_dfs
from pathlib import Path

ANNOTATIONS_PATH = Path('../corpora/annotations')
OUT_DIR = Path('corpus_data')

aggregate_annotation_dfs(ANNOTATIONS_PATH, OUT_DIR)

In [ ]:
from harmonic_inference.data.corpus_reading import read_dump
from pathlib import Path

files_df = read_dump(Path('corpus_data', 'files.tsv'), index_col=0)
measures_df = read_dump(Path('corpus_data', 'measures.tsv'))
chords_df = read_dump(Path('corpus_data', 'chords.tsv'))
notes_df = read_dump(Path('corpus_data', 'notes.tsv'))

In [ ]:
from harmonic_inference.utils import corpus_utils as cu
import importlib
importlib.reload(cu)
import time
import logging
logging.basicConfig(filename='parsing.log', level=logging.INFO, filemode='w')

t = time.time()
# Remove measure repeats
if isinstance(measures_df.iloc[0].next, tuple):
    measures_df = cu.remove_repeats(measures_df, remove_unreachable=True)
print(f'Remove measures: {time.time() - t}')

t = time.time()
# Remove unmatched
notes_df = cu.remove_unmatched(notes_df, measures_df)
chords_df = cu.remove_unmatched(chords_df, measures_df)
chords_df = chords_df.drop(chords_df.loc[(chords_df.numeral == '@none') | chords_df.numeral.isnull()].index)
print(f'Remove unmatched: {time.time() - t}')

t = time.time()
# Add offsets
if not all([column in notes_df.columns for column in ['offset_beat', 'offset_mc']]):
    notes_df = cu.add_note_offsets(notes_df, measures_df)
print(f'Add note offsets: {time.time() - t}')

t = time.time()
# Merge ties
notes_df = cu.merge_ties(notes_df)
print(f'Merge ties: {time.time() - t}')

t = time.time()
# Add chord metrical info
chords_df = cu.add_chord_metrical_data(chords_df, measures_df)
print(f'Add chords metrical info: {time.time() - t}')

In [ ]:
notes_df

In [ ]:
files_df

In [ ]:
chords_df

In [ ]:
measures_df

In [ ]:
import harmonic_inference.data.piece as piece
import harmonic_inference.utils.harmonic_utils as hu
from tqdm import tqdm


import logging
logging.basicConfig(filename='parsing.log', level=logging.INFO, filemode='w')

import importlib
importlib.reload(piece)
importlib.reload(hu)

for i in tqdm(files_df.index):
    logging.info(f"Parsing file id {i}")
    try:
        chords = chords_df.loc[i]
        notes = notes_df.loc[i]
        measures = measures_df.loc[i]
    except BaseException as e:
        logging.error(f"Error getting data for index {i}: {e}")
        continue

    try:
        score_piece = piece.ScorePiece(notes, chords, measures)
    except BaseException as e:
        logging.error(f"Error parsing index {i}: {e}")
        continue

In [ ]:
chords_df.loc[94].loc[chords_df.loc[94].mc == 71]

In [ ]:
chords_df.loc[102]

In [ ]:
import harmonic_inference_data as hid
import importlib
importlib.reload(hid)

datasets = {'global': {},
            'local': {},
            'none': {}}

datasets['global']['train'], datasets['global']['valid'], datasets['global']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='globalkey_811split', make_dfs=True, transpose_global=True
)

datasets['local']['train'], datasets['local']['valid'], datasets['local']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='localkey_811split', make_dfs=True, transpose_local=True
)

datasets['none']['train'], datasets['none']['valid'], datasets['none']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='811split', make_dfs=True
)

In [ ]:
import harmonic_inference_models as him
import harmonic_utils
import importlib
importlib.reload(him)
from ablation import get_masks_and_names
import torch
import os

masks, mask_names = get_masks_and_names()
dir_name = 'results'

data = 'local'
index = -1
prefix = (data + '_') if data != 'none' else ''

mask = torch.tensor(masks[index])
mask_name = mask_names[index]
checkpoint_dir = os.path.join(dir_name, prefix + mask_name)
log_file_name = os.path.join(dir_name, prefix + mask_name + '.log')

model = him.MusicScoreModel(len(datasets[data]['test'][0]['notes'][0]), len(harmonic_utils.CHORD_TYPES) * 12,
                            dropout=0.2, input_mask=mask)

print(mask_name)
print(checkpoint_dir)

In [ ]:
import os
import model_trainer
import importlib
importlib.reload(model_trainer)

from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

optimizer = Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001)
criterion = CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
schedule_var = 'valid_loss'

trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

In [ ]:
trainer.train()

In [ ]:
trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

loss, acc, outputs, labels = trainer.evaluate(valid=False)
print(loss, acc)

In [ ]:
import eval_utils as eu
import harmonic_utils as hu
import matplotlib.pyplot as plt

label_strings = hu.get_one_hot_labels()
conf_mat = eu.get_conf_mat(labels, outputs)

plt.figure(figsize=(30,30))
plt.imshow(conf_mat, interpolation='none')
plt.colorbar()
plt.xticks(ticks=list(range(len(label_strings))), labels=label_strings, rotation=90, fontsize=10)
plt.yticks(ticks=list(range(len(label_strings))), labels=label_strings, fontsize=10)
plt.show()

In [ ]:
import eval_utils as eu

correct, incorrect = eu.get_correct_and_incorrect_indexes(labels, outputs)
print('Correct: ' + str(len(correct)))
print('Incorrect: ' + str(len(incorrect)))

In [ ]:
import eval_utils as eu
    
eu.print_result(incorrect[0], labels, outputs, limit=10, prob=False)

In [ ]:
import eval_utils as eu

chord, onset_notes, all_notes = eu.get_input_df_rows(incorrect[0], datasets[data]['test'])

print(chord)
print("USED NOTES:")
print(onset_notes)
print()
print("ALL NOTES:")
print(all_notes)

In [ ]:
import matplotlib.pyplot as plt
import eval_utils as eu

correct_ranks, indexes_by_rank = eu.get_correct_ranks(labels, outputs)
    
plt.figure(figsize=(30,30))
plt.bar(range(len(outputs[0])), [len(indexes) for indexes in indexes_by_rank])

In [ ]:
import eval_utils as eu
import importlib
importlib.reload(eu)

eval_df = eu.get_eval_df(labels, outputs, datasets[data]['test'])
eval_df

In [ ]:
import ablation
import importlib
importlib.reload(ablation)

dfs = ablation.load_all_ablated_dfs(directory='results', prefix=prefix[:-1] if len(prefix) > 0 else None)
_, mask_names = ablation.get_masks_and_names()

In [ ]:
import pandas as pd
import os

logs = []
for mask_name in mask_names:
    logs.append(pd.read_csv(os.path.join(os.path.join('results', prefix + mask_name + '.log'))))

In [ ]:
for df, log, mask_name in zip(dfs, logs, mask_names):
    print(f"{mask_name} Acc: {100 * df.correct.sum() / len(df)}")
    print(log.iloc[-1])

In [ ]:
import eval_utils as eu

global_df = eu.load_eval_df('results/global_no_ablation.csv')
local_df = eu.load_eval_df('results/local_no_ablation.csv')
none_df = eu.load_eval_df('results/no_ablation.csv')

In [ ]:
global_df

In [ ]:
global_counts = global_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
local_counts = local_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
none_counts = none_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.scatter(global_counts['count'], global_counts['mean'], color='red', label='Global key')
plt.scatter(local_counts['count'], local_counts['mean'], color='blue', label='Local key')
plt.scatter(none_counts['count'], none_counts['mean'], color='yellow', label='No transposition')
plt.title('Global key transposed')
plt.xlabel('Count')
plt.ylabel('Accuracy')
plt.legend()
plt.show()